In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
# Cargar los datos
steam_games = pd.read_csv('C:/Users/delfi/Downloads/delfina local/PI MLOps - STEAM - DELFINA/datasets/steam_games.csv', usecols=['genres', 'tags', 'id', 'title'])
df_steam_games = pd.DataFrame(steam_games)

# Eliminar filas con valores faltantes
df_steam_games = df_steam_games.dropna(subset=['genres'])

# Establecer la columna 'id' como el índice del DataFrame
df_steam_games.set_index('id', inplace=True)

# Preprocesamiento y combinación de datos relevantes
df_steam_games['combined_features'] = df_steam_games['genres'].fillna('') + ' ' + df_steam_games['tags'].fillna('') + ' ' + df_steam_games['title'].fillna('')

# ambio el type para reducir uso de memoria
df_steam_games[['genres', 'title', 'tags', 'combined_features']] = df_steam_games[['genres', 'title', 'tags', 'combined_features']].astype('category')

# Muestra aleatoria para reducir el tamaño del DataFrame 
steam_games_sample = df_steam_games.sample(frac=0.3, random_state=42)

del df_steam_games

In [11]:
steam_games_sample[['genres', 'title', 'tags', 'combined_features']] = steam_games_sample[['genres', 'title', 'tags', 'combined_features']].astype('category')
steam_games_sample.index = steam_games_sample.index.astype(np.int16)

In [16]:
steam_games_sample.head()


,genres,title,tags,combined_features
id,,,,
292990.0,Strategy,NOBUNAGA'S AMBITION: Souzou (Traditional Chine...,"Strategy, Historical","Strategy Strategy, Historical NOBUNAGA'S AMBIT..."
446760.0,"Action, Indie",Neighborhorde,"Action, Indie, Zombies, Survival, Co-op, 4 Pla...","Action, Indie Action, Indie, Zombies, Survival..."
659750.0,"Action, Adventure, Indie",Reversed Dreamland,"Indie, Adventure, Action, 2D, Platformer, Sing...","Action, Adventure, Indie Indie, Adventure, Act..."
492890.0,"Casual, Free to Play, Indie",Crusaders of the Lost Idols - Epic Starter Pack,"Indie, Casual, Free to Play","Casual, Free to Play, Indie Indie, Casual, Fre..."
234370.0,"Action, Adventure, Indie",Shattered Haven,"Action, Adventure, Indie, Puzzle, Zombies, 2D","Action, Adventure, Indie Action, Adventure, In..."


In [18]:
# Creación de la matriz TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(steam_games_sample['combined_features'])

# Calcular la similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [21]:
def get_recommendations(id, cosine_sim=cosine_sim, df=steam_games_sample):
    """
    Obtiene recomendaciones de juegos similares al producto ingresado mediante su ID.

    Parámetros:
    id (int): ID del producto para el cual se desean obtener recomendaciones.
    cosine_sim (array): Matriz de similitud del coseno entre los juegos.
    df (DataFrame): DataFrame con los datos de juegos y características relevantes.

    Retorna:
    pandas.Series o str: Una serie de títulos de juegos recomendados si se encuentran datos para el ID proporcionado.
                         Si no se encuentran datos para el ID, retorna un mensaje indicando que no se encontraron datos.
    """
    # Verificar si el ID del producto está presente en los índices
    if id not in df.index:
        return "No se encontraron datos para este ID de producto."
    
    # Obtener el índice del producto según el ID ingresado
    idx = df.index.get_loc(id)

    # Calcular la similitud entre el producto ingresado y los demás productos
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 juegos similares
    
    # Obtener los índices de los juegos similares
    game_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de los juegos similares encontrados
    return df['title'].iloc[game_indices]

In [22]:
# Obtener recomendaciones para el producto con ID 292990.0
recomendaciones = get_recommendations(292990.0)

# Verificar si hay datos disponibles para el ID proporcionado
if isinstance(recomendaciones, pd.Series):  # Comprobar si es un objeto tipo Series
    print(recomendaciones)  # Títulos de juegos recomendados
else:
    print("No se encontraron datos para este ID de producto.")

id
298771.0    Nobunaga's Ambition: Souzou - "Nobunaga's Ambi...
440914.0    NOBUNAGA'S AMBITION: Souzou SR - “Nobunaga's A...
297892.0    Nobunaga's Ambition: Souzou - Scenario Itsukus...
297895.0    Nobunaga's Ambition: Souzou - Scenario Shingen...
297894.0    Nobunaga's Ambition: Souzou - Scenario Dokugan...
Name: title, dtype: category
Categories (28827, object): ['! That Bastard Is Trying To Steal Our Gold !', '"Barely Attuned Magic Thingy" Staff', '"Glow Ball" - The billiard puzzle game', '"Just Another Day" - Seduce Me Otome CD', ..., '４人打ちアクション麻雀 / ACTION MAHJONG', '＜/reality＞', '＜/reality＞ Original Soundtrack', '🔴 Circles']
